# Model Training and Fine-tuning

In [3]:
%load_ext autoreload
%autoreload 2

In [21]:
import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import pandas as pd
from glob import glob

REPO_ROOT = "/home/mdorosan/2023/cv-toolkit"
META_PATH = os.path.join(REPO_ROOT, "metadata/kvasir-capsule.csv")
DATA_ROOT = os.path.join(REPO_ROOT, "datasets/kvasir-capsule")

# update with experiment name
EXP_PATH = os.path.join(
    REPO_ROOT, 
    "tutorials/tensorflow_notebooks/classification/sample") 
os.makedirs(EXP_PATH, exist_ok=True)

import sys
sys.path.append(REPO_ROOT)

In [91]:
from cvtoolkit import explore
from cvtoolkit.tensorflow.classification._model import CustomClassifier
import cvtoolkit.tensorflow.classification._config as CONFIG
import cvtoolkit.tensorflow.classification._paths as PATHS
import cvtoolkit.tensorflow.classification._utils as UTILS

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import (
    LearningRateScheduler, CSVLogger, ModelCheckpoint, ReduceLROnPlateau,
)
from tensorflow.keras import (losses, metrics, optimizers)

BASE_MODEL = "ResNet50"
DATAGEN_CONFIG = {     
    'horizontal_flip': True, 
    'brightness_range': (0.85, 1.05),
}

FLOW_CONFIG = {
    'x_col' : "image_path",
    'validate_filenames' : False,
    'seed' : 42,
    'target_size' : (224, 224), 
    'color_mode' : 'rgb',
    'class_mode' : 'categorical', # binary
    'interpolation' : 'bilinear',    
    'batch_size' : 64,
}

FIT_CONFIG = {
    'shuffle' : True,
    'verbose' : 1,    
}

BASE_CONFIG = {
    "include_top": False,
    "input_shape": (*FLOW_CONFIG['target_size'], 3),
    "pooling" : "max",
}
OPTIMIZER = CONFIG.OPTIMIZER_DICT['Adam']
CALLBACKS = [
    # LearningRateScheduler(lambda epoch: 1e-3 * 0.9 ** epoch),
    CSVLogger(os.path.join(REPO_ROOT, EXP_PATH, 'trainlog.csv'), append=False),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=5,
        verbose=1,
        mode='min',
        min_delta=0.0001,
        cooldown=0,
        min_lr=1e-8,
    ),
    # add model checkpoint
]
TRAIN_EPOCHS = 50

FT_CALLBACKS = [
    LearningRateScheduler(
        lambda epoch: 1e-4 * 0.2 ** epoch - TRAIN_EPOCHS),
    CSVLogger(os.path.join(REPO_ROOT, EXP_PATH, 'log.csv'), append=False),
    # add model checkpoint
]
FT_EPOCHS = 10

In [67]:
# load from directory
paths = glob(os.path.join(DATA_ROOT, '*', '*'))


rows = []
for path in paths:
    img_meta = UTILS.path_parser(path)
    rows.append(img_meta)

metadata = pd.DataFrame(rows)
TARGET_KEY = "target" # used to stratify and get y
GROUP_KEY = "case_id" # used for grouped splits


# add some filtering here if necessary
use_classes = ["Normal clean mucosa", "Reduced mucosal view"]
metadata = metadata.loc[metadata[TARGET_KEY].isin(use_classes)]


# from notebook 2
from sklearn.model_selection import GroupShuffleSplit

X, y = metadata.drop(columns=[TARGET_KEY]), metadata[TARGET_KEY]
groups = metadata[GROUP_KEY]
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

for i, (train_index, val_index) in enumerate(gss.split(X, y, groups)):
    TRAIN = metadata.iloc[train_index]
    VAL = metadata.iloc[val_index]

CLASSES = y.unique().tolist()

In [104]:
# init preprocessing function
preprocessing_function = CONFIG.BASE_PREPROCESSOR[BASE_MODEL]

# initialize data generators
train_datagen = ImageDataGenerator(
    **DATAGEN_CONFIG,
    preprocessing_function=preprocessing_function,
)

CLASS_WEIGHTS = UTILS.get_class_weights(
    class_weight='balanced',
    classes=y.unique(),
    y=y,
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocessing_function, 
)

train_dataset = train_datagen.flow_from_dataframe(
    dataframe=TRAIN,
    directory=DATA_ROOT,
    y_col=TARGET_KEY,
    classes=CLASSES,
    **FLOW_CONFIG,
)

val_dataset = test_datagen.flow_from_dataframe(
    dataframe=VAL,
    directory=DATA_ROOT,
    y_col=TARGET_KEY,
    classes=CLASSES,
    **FLOW_CONFIG,
)

Found 30318 non-validated image filenames belonging to 2 classes.
Found 6926 non-validated image filenames belonging to 2 classes.


In [106]:
model = CustomClassifier(
    base=BASE_MODEL, 
    **BASE_CONFIG,
)
# model.build((None, *BASE_CONFIG["input_shape"]))
# model.summary()

## Initial Training

In [107]:
# model.base.trainable = False
model.compile(optimizer=OPTIMIZER, **CONFIG.COMPILE_PARAMS)
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=TRAIN_EPOCHS,
    class_weight=CLASS_WEIGHTS,
    callbacks=CALLBACKS,
    **FIT_CONFIG,
)

Epoch 1/50
  3/474 [..............................] - ETA: 1:19:46 - loss: 5.5393 - AUC_PR: 0.4753 - AUC_ROC: 0.4653

KeyboardInterrupt: 

## Fine-tuning

In [ ]:
PCT_FT_LAYERS = 0.20
N = int(np.ceil(NUM_TRAINABLE * PCT_FT_LAYERS))

In [ ]:
# set ALL layers to trainable
model.base.trainable = True

# leave last N layers as trainable
for layer in model.base.layers[:-N]:
    if layer.get_weights():
        layer.trainable = False

In [ ]:
model.compile(optimizer=OPTIMIZER, **config.COMPILE_PARAMS)
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=FT_EPOCHS,
    class_weight=CLASS_WEIGHTS,
    callbacks=FT_CALLBACKS,
    **FIT_CONFIG,
)